In [1]:
import pandas as pd  
from tqdm import tqdm
import numpy as np

import bs4
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import pandas as pd 
from tqdm import tqdm
from random import *
import time 
import pandas as pd 
import requests

from bs4 import BeautifulSoup
import json
import re
import sys

## Import DATA

In [2]:
demo_dev = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_dev/news.tsv', delimiter='\t', header=None)
demo_tr = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_train/news.tsv', delimiter='\t', header=None)
large_dev = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDlarge_dev/news.tsv', delimiter='\t', header=None)
large_tr = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDlarge_train/news.tsv', delimiter='\t', header=None)

In [213]:
print('demo_dev 데이터:', demo_dev.shape)
print('demo_tr 데이터:', demo_tr.shape)

demo_dev 데이터: (18723, 8)
demo_tr 데이터: (26740, 8)


## Util Functions

In [161]:
def nurak_data(df, nid_lst):
    dt = df.loc[df[0].isin(nid_lst)]
    dt_urls = dt[5].tolist()
    return dt_urls, dt

In [162]:
def date_crawler(nurak_url_lst):
    # Don't open browser 
    options = Options()
    options.add_argument("--headless")

    unparseable_urls = [] 
    dates_lst = []
    urls_lst = []

    # 링크 전처리 
    urls = []
    for url in nurak_url_lst:
        stripped_url = url.replace('"', '').strip()
        urls.append(stripped_url)

    # 크롤링 
    print('crawling date start.')
    start_time = int(time.time())

    # start web browser 
    chrome_browser = webdriver.Chrome(executable_path='/Users/baeyuna/Documents/chromedriver', options=options)
    with chrome_browser as browser:
        print("bottleneck .... Program will open chrome ... takes ages ...")
        for url in tqdm(urls):
            try:
                chrome_browser.get(url)
                try:
                    date_element = chrome_browser.find_elements_by_tag_name('time')[0]
                    date = date_element.get_attribute('datetime')
                    dates_lst.append(date)
                    print(f"Yeee!!! date: {date}") 

                    urls_lst.append(url)
                except:
                    print('noooooo!!! parsing {}'.format(url))
                    dates_lst.append(None)
                    urls_lst.append(url)
                    unparseable_urls.append(url)
            except:
                print('noooooo!!! parsing {}'.format(url))
                dates_lst.append(None)
                urls_lst.append(url)
                unparseable_urls.append(url)
        # close web browser to close session and avoid socket errors
        browser.quit()

    print('-------Seconds: %s--------' % (time.time() - start_time))
    df = pd.DataFrame({'date':dates_lst, 'url': urls_lst})
    print(df.shape)
    return df

In [163]:
def crawler(nurak_url_list):
    # 추가 수집 데이터 
    titles = []   
    contents = []
    images = [] 

    print('crawling start.')
    start_time = int(time.time())

    # 링크 전처리 
    urls = []
    for url in nurak_url_list:
        stripped_url = url.replace('"', '').strip()
        urls.append(stripped_url)

    url_except =[]
    num = 0
    for url in tqdm(urls):
        num += 1
        try:
            req = requests.get(url)
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')

            # title
            if soup.find('h1').text.strip() is not None:
                title = soup.find('h1').text.strip()
                titles.append(title)
            elif soup.find('header').text.strip() is not None:
                title = soup.find('header').text.strip()
                titles.append(title)
            else:
                titles.append(None)
            print('title :{}'.format(title))

            # content
            if soup.find_all('p') is not None:
                content = []
                for el in soup.find_all('p'):
                    content.append(el.get_text()) 
                contents.append(' '.join(content))
            elif soup.find('h2').text.strip() is not None:
                content = soup.find('h2').text.strip()
                contents.append(content)
            else:
                contents.append(url)

            # image
            try:
                image = soup.find('img').get('src')
                images.append(image)
            except:
                images.append(None)


        except:
            print('nooooooo!!!! parsing url: {}'.format(url))
            url_except.append(url)

        # 길이 맞추기 
        if len(titles) != num:
            titles.append(None)
        if len(contents) != num:
            contents.append(None)
        if len(images) != num:
            images.append(None)
            
    print('-------Seconds: %s--------' % (time.time() - start_time))
    print(len(dt_urls))
    print(len(titles))
    print(len(contents))
    print(len(images))

    df2 = pd.DataFrame({'url' : dt_urls,
                    'Title':titles,
                    'Content': contents,
                    'Image': images})

    # 주소 이전시 
    for i in range(len(df2)):
        if df2.loc[i, 'Title'] == 'Whoops!':
            df2.loc[i, 'Title'] = np.nan 
            df2.loc[i, 'Content'] = np.nan 
            df2.loc[i, 'Image'] = np.nan 

    print(df2.shape)
    return df2

In [176]:
def after_processing(df_date, df2, dt):
    
    # 날짜, 제목, 내용, 이미지 합치기 
    demo_tr_cr = pd.merge(df_date, df2, on='url')
    print('누락 데이터 수집한 shape')
    print(demo_tr_cr.shape)
    
    # 칼럼 명 맞추고 
    demo_tr_cr = demo_tr_cr.rename(columns={'url':5})
    
    dt = pd.merge(demo_tr_cr, dt, on=5)
    print('누락 데이터 수집한 후 칼럼 증가!! shape')
    print(dt.shape)
    

    return dt

In [165]:
def check_nurak_num(data, integrated_df_path):
    print('원래 데이터 demo_tr:', data.shape)

    integrated_df = pd.read_csv('{}.csv'.format(integrated_df_path))
    print('수집된 데이터 붙인 demo_tr:', integrated_df.shape)

    print('-'*40)
    print('누락 데이터 수:', data.shape[0] - integrated_df.shape[0])
    return 

In [166]:
def final(data, dt, integrated_df_path):
    print('원래 데이터 demo_tr:', data.shape)

    integrated_df = pd.read_csv('{}.csv'.format(integrated_df_path))
    dt = dt.rename(columns={5:'5'})
    dt = dt.rename(columns={0:'0'})
    dt = dt.rename(columns={1:'1'})
    dt = dt.rename(columns={2:'2'})
    dt = dt.rename(columns={3:'3'})
    dt = dt.rename(columns={4:'4'})
    dt = dt.rename(columns={6:'6'})
    dt = dt.rename(columns={7:'7'})
    dt = dt.rename(columns={'date':'Date'})
    dt = dt[integrated_df.columns.tolist()]
    
    df = pd.concat([dt, integrated_df])
    print('추가 수집 완료 최종본:', df.shape)
    return df

In [167]:
def save_df(dt, save_path):
    dt.to_csv('{}/integrated_news.csv'.format(save_path), index=False)
    print('save {}/integrated_news.csv'.format(save_path))
    return 

In [168]:
def crawling_ratio(dt, column_name):
    print('전체 {} 행 수:'.format(column_name) , dt[column_name].shape[0])
    print('nan인 {} 행 수:'.format(column_name), dt.loc[dt[column_name].isnull()].shape[0])
    print('-'*30)

    print('{} 크롤링 가능 비율:'.format(column_name), 100 - (dt.loc[dt[column_name].isnull()].shape[0] / dt[column_name].shape[0])*100)

# 실행

## 1. demo_tr

In [177]:
dt_urls, dt = nurak_data(demo_tr, ['N22921', 'N3052'])

print('----누락 url list-----')
print(dt_urls)

print()
print('----누락 원본 데이터-----')
dt 

----누락 url list-----
["https://www.msn.com/en-us/news/newspolitics/we'd-arrest-trump-for-fifth-ave-shooting,-nyc-mayor-says/ar-AAJin8v?ocid=chopendata", "https://www.msn.com/en-us/tv/tv-celebrity/sharon-osbourne-admits-she-'can-hardly-feel'-her-mouth-after-new-facelift/ar-AAJe15t?ocid=chopendata"]

----누락 원본 데이터-----


,0,1,2,3,4,5,6,7
8751,N22921,news,newspolitics,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",NYPD cops would put President Donald Trump in ...,https://www.msn.com/en-us/news/newspolitics/we...,"[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid...","[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid..."
12489,N3052,tv,tv-celebrity,Sharon Osbourne Admits She 'Can Hardly Feel' H...,Sharon Osbourne Admits She 'Can Hardly Feel' M...,https://www.msn.com/en-us/tv/tv-celebrity/shar...,"[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi...","[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi..."


 날짜 수집 

In [171]:
df_date = date_crawler(dt_urls)
df_date

crawling date start.


  0%|          | 0/2 [00:00<?, ?it/s]

bottleneck .... Program will open chrome ... takes ages ...


 50%|█████     | 1/2 [00:01<00:01,  1.87s/it]

Yeee!!! date: 2019-10-24T18:20:00.000Z


100%|██████████| 2/2 [00:03<00:00,  1.77s/it]

Yeee!!! date: 2019-10-23T13:01:53.000Z
-------Seconds: 6.142351865768433--------
(2, 2)


,date,url
0,2019-10-24T18:20:00.000Z,https://www.msn.com/en-us/news/newspolitics/we...
1,2019-10-23T13:01:53.000Z,https://www.msn.com/en-us/tv/tv-celebrity/shar...


이미지, 제목, 컨텐츠

In [172]:
df2 = crawler(dt_urls)
df2

  0%|          | 0/2 [00:00<?, ?it/s]

crawling start.


 50%|█████     | 1/2 [00:00<00:00,  2.30it/s]

title :We'd Arrest Trump For Fifth Ave Shooting, NYC Mayor Says


100%|██████████| 2/2 [00:00<00:00,  2.33it/s]

title :Sharon Osbourne Admits She 'Can Hardly Feel' Her Mouth After New Facelift
-------Seconds: 1.0142531394958496--------
2
2
2
2
(2, 4)


,url,Title,Content,Image
0,https://www.msn.com/en-us/news/newspolitics/we...,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",NEW YORK — He might give up his freedom even i...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/tv/tv-celebrity/shar...,Sharon Osbourne Admits She 'Can Hardly Feel' H...,Sharon Osbourne isn’t afraid to speak candidly...,//static-entertainment-eas-s-msn-com.akamaized...


In [178]:
dt = after_processing(df_date, df2, dt)
dt

누락 데이터 수집한 shape
(2, 5)
누락 데이터 수집한 후 칼럼 증가!! shape
(2, 12)


,date,5,Title,Content,Image,0,1,2,3,4,6,7
0,2019-10-24T18:20:00.000Z,https://www.msn.com/en-us/news/newspolitics/we...,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",NEW YORK — He might give up his freedom even i...,//static-entertainment-eas-s-msn-com.akamaized...,N22921,news,newspolitics,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",NYPD cops would put President Donald Trump in ...,"[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid...","[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid..."
1,2019-10-23T13:01:53.000Z,https://www.msn.com/en-us/tv/tv-celebrity/shar...,Sharon Osbourne Admits She 'Can Hardly Feel' H...,Sharon Osbourne isn’t afraid to speak candidly...,//static-entertainment-eas-s-msn-com.akamaized...,N3052,tv,tv-celebrity,Sharon Osbourne Admits She 'Can Hardly Feel' H...,Sharon Osbourne Admits She 'Can Hardly Feel' M...,"[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi...","[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi..."


In [179]:
check_nurak_num(demo_tr, '/Users/baeyuna/Documents/SNU_DLab/Data/demo_tr/integrated_news')

원래 데이터 demo_tr: (26740, 8)
수집된 데이터 붙인 demo_tr: (26738, 12)
----------------------------------------
누락 데이터 수: 2


In [180]:
dt = final(demo_tr ,dt, '/Users/baeyuna/Documents/SNU_DLab/Data/demo_tr/integrated_news')
dt.head()

원래 데이터 demo_tr: (26740, 8)
추가 수집 완료 최종본: (26740, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,https://www.msn.com/en-us/news/newspolitics/we...,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",2019-10-24T18:20:00.000Z,//static-entertainment-eas-s-msn-com.akamaized...,NEW YORK — He might give up his freedom even i...,N22921,news,newspolitics,"We'd Arrest Trump For Fifth Ave Shooting, NYC ...",NYPD cops would put President Donald Trump in ...,"[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid...","[{""Label"": ""Fifth Avenue"", ""Type"": ""S"", ""Wikid..."
1,https://www.msn.com/en-us/tv/tv-celebrity/shar...,Sharon Osbourne Admits She 'Can Hardly Feel' H...,2019-10-23T13:01:53.000Z,//static-entertainment-eas-s-msn-com.akamaized...,Sharon Osbourne isn’t afraid to speak candidly...,N3052,tv,tv-celebrity,Sharon Osbourne Admits She 'Can Hardly Feel' H...,Sharon Osbourne Admits She 'Can Hardly Feel' M...,"[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi...","[{""Label"": ""Sharon Osbourne"", ""Type"": ""P"", ""Wi..."
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
1,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]
2,https://www.msn.com/en-us/health/weightloss/th...,NaN,NaN,NaN,NaN,N25099,health,weightloss,These Simple Diet Changes Helped This Guy Lose...,The part of me that knew I had to make changes...,[],[]


In [29]:
# save_df(dt, '/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_train')

save /Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_train integrated_news


In [181]:
crawling_ratio(dt, 'Title')
crawling_ratio(dt, 'Date')
crawling_ratio(dt, 'Image')
crawling_ratio(dt, 'Content')

전체 Title 행 수: 26740
nan인 Title 행 수: 1090
------------------------------
Title 크롤링 가능 비율: 95.92370979805534
전체 Date 행 수: 26740
nan인 Date 행 수: 1666
------------------------------
Date 크롤링 가능 비율: 93.7696335078534
전체 Image 행 수: 26740
nan인 Image 행 수: 5183
------------------------------
Image 크롤링 가능 비율: 80.6170531039641
전체 Content 행 수: 26740
nan인 Content 행 수: 3719
------------------------------
Content 크롤링 가능 비율: 86.09199700822738


## 2. demo_dev

In [182]:
dt_urls, dt = nurak_data(demo_dev, ['N9896', 'N23203', 'N4199'])

print('----누락 url list-----')
print(dt_urls)

print()
print('----누락 원본 데이터-----')
dt 

----누락 url list-----
["https://www.msn.com/en-us/sports/football_nfl/marcus-peters'-fit-with-ravens-looks-to-be-better-than-his-stint-with-rams/ar-AAIUGeH?ocid=chopendata", 'https://www.msn.com/en-us/news/newsus/the-fundamentalist-religious-community-that-lives-in-the-throes-of-a-drug-war/vi-AAJXUWc?ocid=chopendata', "https://www.msn.com/en-us/news/newscrime/missing-college-student's-parents-meet-with-mother-of-natalee-holloway/ar-AAJYeVW?ocid=chopendata"]

----누락 원본 데이터-----


,0,1,2,3,4,5,6,7
4802,N9896,sports,football_nfl,Marcus Peters' fit with Ravens looks to be bet...,NaN,https://www.msn.com/en-us/sports/football_nfl/...,"[{""Label"": ""Los Angeles Rams"", ""Type"": ""O"", ""W...",[]
6611,N23203,news,newsus,The fundamentalist religious community that li...,The three mothers and six children killed in a...,https://www.msn.com/en-us/news/newsus/the-fund...,[],"[{""Label"": ""Mexican Drug War"", ""Type"": ""E"", ""W..."
10368,N4199,news,newscrime,Missing college student's parents meet with mo...,"Investigators did not call the man a suspect, ...",https://www.msn.com/en-us/news/newscrime/missi...,"[{""Label"": ""Disappearance of Natalee Holloway""...",[]


In [183]:
df_date = date_crawler(dt_urls)
df_date

crawling date start.


  0%|          | 0/3 [00:00<?, ?it/s]

bottleneck .... Program will open chrome ... takes ages ...


 33%|███▎      | 1/3 [00:01<00:03,  1.69s/it]

Yeee!!! date: 2019-10-17T03:22:27.000Z


 67%|██████▋   | 2/3 [00:02<00:01,  1.48s/it]

noooooo!!! parsing https://www.msn.com/en-us/news/newsus/the-fundamentalist-religious-community-that-lives-in-the-throes-of-a-drug-war/vi-AAJXUWc?ocid=chopendata


100%|██████████| 3/3 [00:03<00:00,  1.25s/it]

Yeee!!! date: 2019-11-07T06:49:00.000Z
-------Seconds: 6.236371040344238--------
(3, 2)


,date,url
0,2019-10-17T03:22:27.000Z,https://www.msn.com/en-us/sports/football_nfl/...
1,None,https://www.msn.com/en-us/news/newsus/the-fund...
2,2019-11-07T06:49:00.000Z,https://www.msn.com/en-us/news/newscrime/missi...


In [184]:
df2 = crawler(dt_urls)
df2

  0%|          | 0/3 [00:00<?, ?it/s]

crawling start.


 67%|██████▋   | 2/3 [00:00<00:00,  4.66it/s]

title :Marcus Peters' fit with Ravens looks to be better than his stint with Rams
nooooooo!!!! parsing url: https://www.msn.com/en-us/news/newsus/the-fundamentalist-religious-community-that-lives-in-the-throes-of-a-drug-war/vi-AAJXUWc?ocid=chopendata


100%|██████████| 3/3 [00:00<00:00,  4.45it/s]

title :Missing college student's parents meet with mother of Natalee Holloway
-------Seconds: 0.9260411262512207--------
3
3
3
3
(3, 4)


,url,Title,Content,Image
0,https://www.msn.com/en-us/sports/football_nfl/...,Marcus Peters' fit with Ravens looks to be bet...,To understand why the Baltimore Ravens made th...,//static-entertainment-eas-s-msn-com.akamaized...
1,https://www.msn.com/en-us/news/newsus/the-fund...,None,None,None
2,https://www.msn.com/en-us/news/newscrime/missi...,Missing college student's parents meet with mo...,\n\n\n\n \n\n< PREVIOUS SLIDE\n\nSLIDE 1 of 3\...,//static-entertainment-eas-s-msn-com.akamaized...


In [186]:
dt = after_processing(df_date, df2, dt)
dt

누락 데이터 수집한 shape
(3, 5)
누락 데이터 수집한 후 칼럼 증가!! shape
(3, 12)


,date,5,Title,Content,Image,0,1,2,3,4,6,7
0,2019-10-17T03:22:27.000Z,https://www.msn.com/en-us/sports/football_nfl/...,Marcus Peters' fit with Ravens looks to be bet...,To understand why the Baltimore Ravens made th...,//static-entertainment-eas-s-msn-com.akamaized...,N9896,sports,football_nfl,Marcus Peters' fit with Ravens looks to be bet...,NaN,"[{""Label"": ""Los Angeles Rams"", ""Type"": ""O"", ""W...",[]
1,None,https://www.msn.com/en-us/news/newsus/the-fund...,None,None,None,N23203,news,newsus,The fundamentalist religious community that li...,The three mothers and six children killed in a...,[],"[{""Label"": ""Mexican Drug War"", ""Type"": ""E"", ""W..."
2,2019-11-07T06:49:00.000Z,https://www.msn.com/en-us/news/newscrime/missi...,Missing college student's parents meet with mo...,\n\n\n\n \n\n< PREVIOUS SLIDE\n\nSLIDE 1 of 3\...,//static-entertainment-eas-s-msn-com.akamaized...,N4199,news,newscrime,Missing college student's parents meet with mo...,"Investigators did not call the man a suspect, ...","[{""Label"": ""Disappearance of Natalee Holloway""...",[]


In [188]:
check_nurak_num(demo_dev, '/Users/baeyuna/Documents/SNU_DLab/Data/demo_dev/integrated_news')

원래 데이터 demo_tr: (18723, 8)
수집된 데이터 붙인 demo_tr: (18720, 12)
----------------------------------------
누락 데이터 수: 3


In [189]:
dt = final(demo_dev, dt, '/Users/baeyuna/Documents/SNU_DLab/Data/demo_dev/integrated_news')
dt.head() 

원래 데이터 demo_tr: (18723, 8)
추가 수집 완료 최종본: (18723, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,https://www.msn.com/en-us/sports/football_nfl/...,Marcus Peters' fit with Ravens looks to be bet...,2019-10-17T03:22:27.000Z,//static-entertainment-eas-s-msn-com.akamaized...,To understand why the Baltimore Ravens made th...,N9896,sports,football_nfl,Marcus Peters' fit with Ravens looks to be bet...,NaN,"[{""Label"": ""Los Angeles Rams"", ""Type"": ""O"", ""W...",[]
1,https://www.msn.com/en-us/news/newsus/the-fund...,None,None,None,None,N23203,news,newsus,The fundamentalist religious community that li...,The three mothers and six children killed in a...,[],"[{""Label"": ""Mexican Drug War"", ""Type"": ""E"", ""W..."
2,https://www.msn.com/en-us/news/newscrime/missi...,Missing college student's parents meet with mo...,2019-11-07T06:49:00.000Z,//static-entertainment-eas-s-msn-com.akamaized...,\n\n\n\n \n\n< PREVIOUS SLIDE\n\nSLIDE 1 of 3\...,N4199,news,newscrime,Missing college student's parents meet with mo...,"Investigators did not call the man a suspect, ...","[{""Label"": ""Disappearance of Natalee Holloway""...",[]
0,https://www.msn.com/en-us/news/world/the-cost-...,NaN,NaN,NaN,NaN,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
1,https://www.msn.com/en-us/health/nutrition/50-...,NaN,NaN,NaN,NaN,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]


In [70]:
# save_df(dt, '/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_dev')

save /Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_dev/integrated_news.csv


In [190]:
crawling_ratio(dt, 'Title')
crawling_ratio(dt, 'Date')
crawling_ratio(dt, 'Image')
crawling_ratio(dt, 'Content')

전체 Title 행 수: 18723
nan인 Title 행 수: 1618
------------------------------
Title 크롤링 가능 비율: 91.35822250707686
전체 Date 행 수: 18723
nan인 Date 행 수: 1731
------------------------------
Date 크롤링 가능 비율: 90.75468674891845
전체 Image 행 수: 18723
nan인 Image 행 수: 3593
------------------------------
Image 크롤링 가능 비율: 80.8096993003258
전체 Content 행 수: 18723
nan인 Content 행 수: 3175
------------------------------
Content 크롤링 가능 비율: 83.04224750307108


## 3. large_tr  

* url 형식이 이상함

In [191]:
dt_urls, dt = nurak_data(large_tr, ['N110434', 'N74207', 'N113363', 'N118418', 'N102010', 'N45635'])

print('----누락 url list-----')
print(dt_urls)

print()
print('----누락 원본 데이터-----')
dt 

----누락 url list-----
['[]', '[]', '[{"Label": "Freddie Kitchens", "Type": "P", "WikidataId": "Q5496752", "Confidence": 1.0, "OccurrenceOffsets": [90], "SurfaceForms": ["Freddie Kitchens"]}, {"Label": "Baker Mayfield", "Type": "P", "WikidataId": "Q14950655", "Confidence": 1.0, "OccurrenceOffsets": [115], "SurfaceForms": ["Mayfield"]}]', '[{"Label": "Forest Park\\u2013DeBaliviere station", "Type": "F", "WikidataId": "Q5469071", "Confidence": 1.0, "OccurrenceOffsets": [36], "SurfaceForms": ["Forest Park Debaliviere"]}]', '[{"Label": "Animal control service", "Type": "U", "WikidataId": "Q3289646", "Confidence": 1.0, "OccurrenceOffsets": [51], "SurfaceForms": ["animal control"]}]', '[{"Label": "Sean Hannity", "Type": "P", "WikidataId": "Q557730", "Confidence": 1.0, "OccurrenceOffsets": [211], "SurfaceForms": ["Sean Hannity"]}, {"Label": "Max Boot", "Type": "P", "WikidataId": "Q3890747", "Confidence": 1.0, "OccurrenceOffsets": [324], "SurfaceForms": ["Max Boot"]}]']

----누락 원본 데이터-----


,0,1,2,3,4,5,6,7
4664,N110434,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,[],NaN,NaN
38162,N74207,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],[],NaN
46236,N113363,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",NaN,NaN
48766,N118418,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...","[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
63749,N102010,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...","[{""Label"": ""Animal control service"", ""Type"": ""...",NaN
100955,N45635,video,news,Transcripts highlight Sean Hannity's political...,We're seeing really disturbing things coming o...,"[{""Label"": ""Sean Hannity"", ""Type"": ""P"", ""Wikid...",NaN,NaN


In [192]:
df_date = date_crawler(dt_urls)
df_date

crawling date start.


100%|██████████| 6/6 [00:00<00:00, 213.86it/s]

bottleneck .... Program will open chrome ... takes ages ...
noooooo!!! parsing []
noooooo!!! parsing []
noooooo!!! parsing [{Label: Freddie Kitchens, Type: P, WikidataId: Q5496752, Confidence: 1.0, OccurrenceOffsets: [90], SurfaceForms: [Freddie Kitchens]}, {Label: Baker Mayfield, Type: P, WikidataId: Q14950655, Confidence: 1.0, OccurrenceOffsets: [115], SurfaceForms: [Mayfield]}]
noooooo!!! parsing [{Label: Forest Park\u2013DeBaliviere station, Type: F, WikidataId: Q5469071, Confidence: 1.0, OccurrenceOffsets: [36], SurfaceForms: [Forest Park Debaliviere]}]
noooooo!!! parsing [{Label: Animal control service, Type: U, WikidataId: Q3289646, Confidence: 1.0, OccurrenceOffsets: [51], SurfaceForms: [animal control]}]
noooooo!!! parsing [{Label: Sean Hannity, Type: P, WikidataId: Q557730, Confidence: 1.0, OccurrenceOffsets: [211], SurfaceForms: [Sean Hannity]}, {Label: Max Boot, Type: P, WikidataId: Q3890747, Confidence: 1.0, OccurrenceOffsets: [324], SurfaceForms: [Max Boot]}]
-------Secon

,date,url
0,None,[]
1,None,[]
2,None,"[{Label: Freddie Kitchens, Type: P, WikidataId..."
3,None,"[{Label: Forest Park\u2013DeBaliviere station,..."
4,None,"[{Label: Animal control service, Type: U, Wiki..."
5,None,"[{Label: Sean Hannity, Type: P, WikidataId: Q5..."


In [193]:
# 차후 df 결합을 위해 추가 
df_date['0'] = ['N110434', 'N74207', 'N113363', 'N118418', 'N102010', 'N45635']
df_date = df_date.drop(columns='url')
df_date

,date,0
0,None,N110434
1,None,N74207
2,None,N113363
3,None,N118418
4,None,N102010
5,None,N45635


In [194]:
df2 = crawler(dt_urls)
df2

100%|██████████| 6/6 [00:00<00:00, 887.43it/s]

crawling start.
nooooooo!!!! parsing url: []
nooooooo!!!! parsing url: []
nooooooo!!!! parsing url: [{Label: Freddie Kitchens, Type: P, WikidataId: Q5496752, Confidence: 1.0, OccurrenceOffsets: [90], SurfaceForms: [Freddie Kitchens]}, {Label: Baker Mayfield, Type: P, WikidataId: Q14950655, Confidence: 1.0, OccurrenceOffsets: [115], SurfaceForms: [Mayfield]}]
nooooooo!!!! parsing url: [{Label: Forest Park\u2013DeBaliviere station, Type: F, WikidataId: Q5469071, Confidence: 1.0, OccurrenceOffsets: [36], SurfaceForms: [Forest Park Debaliviere]}]
nooooooo!!!! parsing url: [{Label: Animal control service, Type: U, WikidataId: Q3289646, Confidence: 1.0, OccurrenceOffsets: [51], SurfaceForms: [animal control]}]
nooooooo!!!! parsing url: [{Label: Sean Hannity, Type: P, WikidataId: Q557730, Confidence: 1.0, OccurrenceOffsets: [211], SurfaceForms: [Sean Hannity]}, {Label: Max Boot, Type: P, WikidataId: Q3890747, Confidence: 1.0, OccurrenceOffsets: [324], SurfaceForms: [Max Boot]}]
-------Seconds

,url,Title,Content,Image
0,[],None,None,None
1,[],None,None,None
2,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",None,None,None
3,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...",None,None,None
4,"[{""Label"": ""Animal control service"", ""Type"": ""...",None,None,None
5,"[{""Label"": ""Sean Hannity"", ""Type"": ""P"", ""Wikid...",None,None,None


In [195]:
# 차후 df 결합을 위해 추가 
df2['0'] = ['N110434', 'N74207', 'N113363', 'N118418', 'N102010', 'N45635']
df2 = df2.drop(columns='url')
df2

,Title,Content,Image,0
0,None,None,None,N110434
1,None,None,None,N74207
2,None,None,None,N113363
3,None,None,None,N118418
4,None,None,None,N102010
5,None,None,None,N45635


In [196]:
def after_processing_strange_url(df_date, df2, dt):
    
    # 날짜, 제목, 내용, 이미지 합치기 
    demo_tr_cr = pd.merge(df_date, df2, on='0')
    print('누락 데이터 수집한 shape')
    print(demo_tr_cr.shape)
    
    # 칼럼 명 맞추고 
    demo_tr_cr = demo_tr_cr.rename(columns={'0':0})
    
    dt = pd.merge(demo_tr_cr, dt, on=0)
    print('누락 데이터 수집한 후 칼럼 증가!! shape')
    print(dt.shape)
    

    return dt

In [197]:
dt = after_processing_strange_url(df_date, df2, dt)
dt

누락 데이터 수집한 shape
(6, 5)
누락 데이터 수집한 후 칼럼 증가!! shape
(6, 12)


,date,0,Title,Content,Image,1,2,3,4,5,6,7
0,None,N110434,None,None,None,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,[],NaN,NaN
1,None,N74207,None,None,None,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],[],NaN
2,None,N113363,None,None,None,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",NaN,NaN
3,None,N118418,None,None,None,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...","[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
4,None,N102010,None,None,None,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...","[{""Label"": ""Animal control service"", ""Type"": ""...",NaN
5,None,N45635,None,None,None,video,news,Transcripts highlight Sean Hannity's political...,We're seeing really disturbing things coming o...,"[{""Label"": ""Sean Hannity"", ""Type"": ""P"", ""Wikid...",NaN,NaN


In [198]:
check_nurak_num(large_tr, '/Users/baeyuna/Documents/SNU_DLab/Data/msn_train')

원래 데이터 demo_tr: (101527, 8)
수집된 데이터 붙인 demo_tr: (101521, 13)
----------------------------------------
누락 데이터 수: 6


In [199]:
def final_strange_url(data, dt, integrated_df_path):
    print('원래 데이터 :', data.shape)

    integrated_df = pd.read_csv('{}.csv'.format(integrated_df_path))
    integrated_df = integrated_df.drop(columns='nids')
    
    dt = dt.rename(columns={5:'5'})
    dt = dt.rename(columns={0:'0'})
    dt = dt.rename(columns={1:'1'})
    dt = dt.rename(columns={2:'2'})
    dt = dt.rename(columns={3:'3'})
    dt = dt.rename(columns={4:'4'})
    dt = dt.rename(columns={6:'6'})
    dt = dt.rename(columns={7:'7'})
    dt = dt.rename(columns={'date':'Date'})
    dt = dt[integrated_df.columns.tolist()]
    
    df = pd.concat([dt, integrated_df])
    print('추가 수집 완료 최종본:', df.shape)
    return df

In [200]:
dt = final_strange_url(large_tr, dt, '/Users/baeyuna/Documents/SNU_DLab/Data/msn_train')
dt.head()

원래 데이터 : (101527, 8)
추가 수집 완료 최종본: (101527, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,[],None,None,None,None,N110434,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,NaN,NaN
1,[],None,None,None,None,N74207,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],NaN
2,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",None,None,None,None,N113363,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,NaN,NaN
3,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...",None,None,None,None,N118418,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
4,"[{""Label"": ""Animal control service"", ""Type"": ""...",None,None,None,None,N102010,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...",NaN


In [144]:
# save_df(dt, '/Users/baeyuna/Documents/SNU_DLab/Data/MINDlarge_train')

save /Users/baeyuna/Documents/SNU_DLab/Data/MINDlarge_train/integrated_news.csv


In [201]:
crawling_ratio(dt, 'Title')
crawling_ratio(dt, 'Date')
crawling_ratio(dt, 'Image')
crawling_ratio(dt, 'Content')

전체 Title 행 수: 101527
nan인 Title 행 수: 6
------------------------------
Title 크롤링 가능 비율: 99.99409024200459
전체 Date 행 수: 101527
nan인 Date 행 수: 104
------------------------------
Date 크롤링 가능 비율: 99.89756419474622
전체 Image 행 수: 101527
nan인 Image 행 수: 20313
------------------------------
Image 크롤링 가능 비율: 79.99251430653915
전체 Content 행 수: 101527
nan인 Content 행 수: 10916
------------------------------
Content 크롤링 가능 비율: 89.24818028701725


## 4. large_dev  
* url 형식이 이상함

In [202]:
dt_urls, dt = nurak_data(large_dev, ['N110434', 'N74207', 'N113363', 'N118418', 'N102010'])

print('----누락 url list-----')
print(dt_urls)

print()
print('----누락 원본 데이터-----')
dt 

----누락 url list-----
['[]', '[]', '[{"Label": "Freddie Kitchens", "Type": "P", "WikidataId": "Q5496752", "Confidence": 1.0, "OccurrenceOffsets": [90], "SurfaceForms": ["Freddie Kitchens"]}, {"Label": "Baker Mayfield", "Type": "P", "WikidataId": "Q14950655", "Confidence": 1.0, "OccurrenceOffsets": [115], "SurfaceForms": ["Mayfield"]}]', '[{"Label": "Forest Park\\u2013DeBaliviere station", "Type": "F", "WikidataId": "Q5469071", "Confidence": 1.0, "OccurrenceOffsets": [36], "SurfaceForms": ["Forest Park Debaliviere"]}]', '[{"Label": "Animal control service", "Type": "U", "WikidataId": "Q3289646", "Confidence": 1.0, "OccurrenceOffsets": [51], "SurfaceForms": ["animal control"]}]']

----누락 원본 데이터-----


,0,1,2,3,4,5,6,7
3932,N110434,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,[],NaN,NaN
31636,N74207,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],[],NaN
38358,N113363,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",NaN,NaN
40467,N118418,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...","[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
52863,N102010,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...","[{""Label"": ""Animal control service"", ""Type"": ""...",NaN


In [203]:
df_date = date_crawler(dt_urls)
df_date

crawling date start.


100%|██████████| 5/5 [00:00<00:00, 362.67it/s]

bottleneck .... Program will open chrome ... takes ages ...
noooooo!!! parsing []
noooooo!!! parsing []
noooooo!!! parsing [{Label: Freddie Kitchens, Type: P, WikidataId: Q5496752, Confidence: 1.0, OccurrenceOffsets: [90], SurfaceForms: [Freddie Kitchens]}, {Label: Baker Mayfield, Type: P, WikidataId: Q14950655, Confidence: 1.0, OccurrenceOffsets: [115], SurfaceForms: [Mayfield]}]
noooooo!!! parsing [{Label: Forest Park\u2013DeBaliviere station, Type: F, WikidataId: Q5469071, Confidence: 1.0, OccurrenceOffsets: [36], SurfaceForms: [Forest Park Debaliviere]}]
noooooo!!! parsing [{Label: Animal control service, Type: U, WikidataId: Q3289646, Confidence: 1.0, OccurrenceOffsets: [51], SurfaceForms: [animal control]}]
-------Seconds: 2.054513931274414--------
(5, 2)


,date,url
0,None,[]
1,None,[]
2,None,"[{Label: Freddie Kitchens, Type: P, WikidataId..."
3,None,"[{Label: Forest Park\u2013DeBaliviere station,..."
4,None,"[{Label: Animal control service, Type: U, Wiki..."


In [204]:
# 차후 df 결합을 위해 추가 
df_date['0'] = ['N110434', 'N74207', 'N113363', 'N118418', 'N102010']
df_date = df_date.drop(columns='url')
df_date

,date,0
0,None,N110434
1,None,N74207
2,None,N113363
3,None,N118418
4,None,N102010


In [205]:
df2 = crawler(dt_urls)
df2

100%|██████████| 5/5 [00:00<00:00, 1128.23it/s]

crawling start.
nooooooo!!!! parsing url: []
nooooooo!!!! parsing url: []
nooooooo!!!! parsing url: [{Label: Freddie Kitchens, Type: P, WikidataId: Q5496752, Confidence: 1.0, OccurrenceOffsets: [90], SurfaceForms: [Freddie Kitchens]}, {Label: Baker Mayfield, Type: P, WikidataId: Q14950655, Confidence: 1.0, OccurrenceOffsets: [115], SurfaceForms: [Mayfield]}]
nooooooo!!!! parsing url: [{Label: Forest Park\u2013DeBaliviere station, Type: F, WikidataId: Q5469071, Confidence: 1.0, OccurrenceOffsets: [36], SurfaceForms: [Forest Park Debaliviere]}]
nooooooo!!!! parsing url: [{Label: Animal control service, Type: U, WikidataId: Q3289646, Confidence: 1.0, OccurrenceOffsets: [51], SurfaceForms: [animal control]}]
-------Seconds: 0.08394598960876465--------
5
5
5
5
(5, 4)


,url,Title,Content,Image
0,[],None,None,None
1,[],None,None,None
2,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",None,None,None
3,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...",None,None,None
4,"[{""Label"": ""Animal control service"", ""Type"": ""...",None,None,None


In [206]:
# 차후 df 결합을 위해 추가 
df2['0'] = ['N110434', 'N74207', 'N113363', 'N118418', 'N102010']
df2 = df2.drop(columns='url')
df2

,Title,Content,Image,0
0,None,None,None,N110434
1,None,None,None,N74207
2,None,None,None,N113363
3,None,None,None,N118418
4,None,None,None,N102010


In [207]:
dt = after_processing_strange_url(df_date, df2, dt)
dt

누락 데이터 수집한 shape
(5, 5)
누락 데이터 수집한 후 칼럼 증가!! shape
(5, 12)


,date,0,Title,Content,Image,1,2,3,4,5,6,7
0,None,N110434,None,None,None,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,[],NaN,NaN
1,None,N74207,None,None,None,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],[],NaN
2,None,N113363,None,None,None,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",NaN,NaN
3,None,N118418,None,None,None,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...","[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
4,None,N102010,None,None,None,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...","[{""Label"": ""Animal control service"", ""Type"": ""...",NaN


In [208]:
check_nurak_num(large_dev, '/Users/baeyuna/Documents/SNU_DLab/Data/msn_dev')

원래 데이터 demo_tr: (72023, 8)
수집된 데이터 붙인 demo_tr: (72018, 13)
----------------------------------------
누락 데이터 수: 5


In [209]:
dt = final_strange_url(large_dev, dt, '/Users/baeyuna/Documents/SNU_DLab/Data/msn_dev')
dt.head()

원래 데이터 : (72023, 8)
추가 수집 완료 최종본: (72023, 12)


,5,Title,Date,Image,Content,0,1,2,3,4,6,7
0,[],None,None,None,None,N110434,lifestyle,lifestylefamilyandrelationships,The 50 Most Common Last Names in America,What's in a name?\thttps://assets.msn.com/labs...,NaN,NaN
1,[],None,None,None,None,N74207,finance,finance-video,The Price You Pay: The spiraling cost of colle...,https://assets.msn.com/labs/mind/AAIKUGl.html,[],NaN
2,"[{""Label"": ""Freddie Kitchens"", ""Type"": ""P"", ""W...",None,None,None,None,N113363,sports,football_nfl,Baker Mayfield injury update: 'No doubt' Brown...,He'll be in better shape physically and he'll ...,NaN,NaN
3,"[{""Label"": ""Forest Park\u2013DeBaliviere stati...",None,None,None,None,N118418,news,newsus,The security has been stepped up': Forest Park...,https://assets.msn.com/labs/mind/AAJncPt.html,"[{""Label"": ""St. Louis"", ""Type"": ""G"", ""Wikidata...",NaN
4,"[{""Label"": ""Animal control service"", ""Type"": ""...",None,None,None,None,N102010,news,newsus,[We] walk dogs and pick up poo for free | Why ...,https://assets.msn.com/labs/mind/AAJUQ4w.html,"[{""Label"": ""Animal control service"", ""Type"": ""...",NaN


In [210]:
# save_df(dt, '/Users/baeyuna/Documents/SNU_DLab/Data/MINDlarge_dev')

In [211]:
crawling_ratio(dt, 'Title')
crawling_ratio(dt, 'Date')
crawling_ratio(dt, 'Image')
crawling_ratio(dt, 'Content')

전체 Title 행 수: 72023
nan인 Title 행 수: 5
------------------------------
Title 크롤링 가능 비율: 99.99305777321133
전체 Date 행 수: 72023
nan인 Date 행 수: 483
------------------------------
Date 크롤링 가능 비율: 99.32938089221499
전체 Image 행 수: 72023
nan인 Image 행 수: 13653
------------------------------
Image 크롤링 가능 비율: 81.04355553087208
전체 Content 행 수: 72023
nan인 Content 행 수: 7176
------------------------------
Content 크롤링 가능 비율: 90.03651611290837
